In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("ner_dataset.csv", encoding="latin1")

In [2]:
data = data.fillna(method="ffill")
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [3]:
words = list(set(data["Word"].values))
n_words = len(words); n_words

35178

In [4]:
def get_sentences(data):
    agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
    sentence_grouped = data.groupby("Sentence #").apply(agg_func)
    return [s for s in sentence_grouped] 

In [5]:
sentences = get_sentences(data)

In [6]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0: # word_prev, word_curr
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [18]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [13]:
! pip install sklearn-crfsuite -i https://app.wistron.com/applogin.aspx

Looking in indexes: https://app.wistron.com/applogin.aspx

ERROR: Could not find a version that satisfies the requirement sklearn-crfsuite (from versions: none)
ERROR: No matching distribution found for sklearn-crfsuite



Could not fetch URL https://app.wistron.com/applogin.aspx/sklearn-crfsuite/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='app.wistron.com', port=443): Max retries exceeded with url: /applogin.aspx/sklearn-crfsuite/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1125)'))) - skipping
Could not fetch URL https://app.wistron.com/applogin.aspx/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='app.wistron.com', port=443): Max retries exceeded with url: /applogin.aspx/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1125)'))) - skipping


In [19]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100)

In [23]:
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
# pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)
try:
    crf.fit(X=X, y=y)
except AttributeError:
    pass
pred = crf.predict(X=X)


In [24]:
from sklearn_crfsuite.metrics import flat_classification_report
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

c:\Users\11004142\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


              precision    recall  f1-score   support

       B-art       0.93      0.73      0.82       402
       B-eve       0.85      0.69      0.76       308
       B-geo       0.91      0.95      0.93     37644
       B-gpe       0.98      0.95      0.97     15870
       B-nat       0.90      0.60      0.72       201
       B-org       0.91      0.85      0.88     20143
       B-per       0.94      0.93      0.94     16990
       B-tim       0.96      0.92      0.94     20333
       I-art       0.94      0.77      0.85       297
       I-eve       0.87      0.67      0.76       253
       I-geo       0.90      0.92      0.91      7414
       I-gpe       0.96      0.69      0.80       198
       I-nat       0.95      0.71      0.81        51
       I-org       0.94      0.93      0.94     16784
       I-per       0.94      0.96      0.95     17251
       I-tim       0.94      0.88      0.91      6528
           O       1.00      1.00      1.00    887908

    accuracy              